In [1]:
import pandas as pd
import sagemaker
import boto3
import numpy as np
import keras
import tensorflow as tf

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Miray\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-tensorflow-chest-scan-image-classification'

# Retrieving the IAM role
try:
    role = sagemaker.get_execution_role()
except Exception as e:
    print("IAM role could not be obtained. A fallback approach is used.")
    iam = boto3.client('iam')
    role_name = "role_name"  # Replace with your IAM role name
    role = iam.get_role(RoleName=role_name)['Role']['Arn']

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sagemaker_session.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Miray\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Miray\AppData\Local\sagemaker\sagemaker\config.yaml


Couldn't call 'get_role' to get Role ARN from role name arn:aws:iam::120644112934:root to get Role path.


IAM role could not be obtained. A fallback approach is used.
sagemaker role arn: arn:aws:iam::120644112934:role/sagemaker-user
sagemaker session region: us-east-1


In [3]:
keras.utils.set_random_seed(42)

In [4]:
!aws s3 cp train/ s3://bucket_name/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/ --recursive

Completed 77.2 KiB/42.1 MiB (2.7 KiB/s) with 478 file(s) remaining
upload: train\adenocarcinoma\000017 (8).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000017 (8).png
Completed 77.2 KiB/42.1 MiB (2.7 KiB/s) with 477 file(s) remaining
Completed 177.6 KiB/42.1 MiB (3.5 KiB/s) with 477 file(s) remaining
upload: train\adenocarcinoma\000020 (6).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000020 (6).png
Completed 177.6 KiB/42.1 MiB (3.5 KiB/s) with 476 file(s) remaining
Completed 268.1 KiB/42.1 MiB (4.0 KiB/s) with 476 file(s) remaining
upload: train\adenocarcinoma\000019 (5).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000019 (5).png
Completed 268.1 KiB/42.1 MiB (4.0 KiB/s) with 475 file(s) remaining
Completed 330.8 KiB/42.1 MiB (4.2 KiB/s) with 475 file(s)

File association not found for extension .py
upload failed: train\adenocarcinoma\000030 (4).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000030 (4).png SSL validation failed for https://sagemaker-us-east-1-120644112934.s3.amazonaws.com/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000030%20%284%29.png TLS/SSL connection has been closed (EOF) (_ssl.c:1131)
upload failed: train\adenocarcinoma\000024 (7).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000024 (7).png Connection was closed before we received a valid response from endpoint URL: "https://sagemaker-us-east-1-120644112934.s3.amazonaws.com/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train/adenocarcinoma/000024%20%287%29.png".
upload failed: train\adenocarcinoma\000027 (4).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO

In [5]:
!aws s3 cp test/ s3://bucket_name/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/ --recursive

Completed 65.0 KiB/9.7 MiB (2.0 KiB/s) with 100 file(s) remaining
upload: test\adenocarcinoma\000008 (10).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/adenocarcinoma/000008 (10).png
Completed 65.0 KiB/9.7 MiB (2.0 KiB/s) with 99 file(s) remaining
Completed 167.5 KiB/9.7 MiB (4.8 KiB/s) with 99 file(s) remaining
upload: test\adenocarcinoma\000009 (7).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/adenocarcinoma/000009 (7).png
Completed 167.5 KiB/9.7 MiB (4.8 KiB/s) with 98 file(s) remaining
Completed 282.4 KiB/9.7 MiB (5.4 KiB/s) with 98 file(s) remaining
upload: test\adenocarcinoma\000132 (7).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/adenocarcinoma/000132 (7).png
Completed 282.4 KiB/9.7 MiB (5.4 KiB/s) with 97 file(s) remaining
Completed 373.2 KiB/9.7 MiB (6.4 KiB/s) with 97 file(s) remaining
upload

File association not found for extension .py
upload failed: test\adenocarcinoma\000013 (8).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/adenocarcinoma/000013 (8).png SSL validation failed for https://sagemaker-us-east-1-120644112934.s3.amazonaws.com/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/adenocarcinoma/000013%20%288%29.png TLS/SSL connection has been closed (EOF) (_ssl.c:1131)
upload failed: test\adenocarcinoma\000153 (5).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test/adenocarcinoma/000153 (5).png An error occurred (RequestTimeout) when calling the PutObject operation (reached max retries: 4): Your socket connection to the server was not read from or written to within the timeout period. Idle connections will be closed.
upload failed: test\adenocarcinoma\000157 (5).png to s3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest

In [3]:
!mkdir code

In [3]:
%cd code

C:\Users\Miray\Desktop\Chest_Scan\code


In [4]:
%%writefile model.py

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense

def get_model():
  model = Sequential()
  model.add(Conv2D(32, (5,5), activation='relu', input_shape=(128,128,3)))
  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.1))

  model.add(Conv2D(64, (5,5), activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.1))

  model.add(Conv2D(128, (5,5), activation='relu'))
  model.add(MaxPool2D(2,2))

  model.add(Conv2D(256, (5,5), activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.1))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(1, activation='sigmoid'))

  return model

Overwriting model.py


In [7]:
%%writefile train.py

import os
import numpy as np
import tensorflow as tf
import argparse
from argparse import ArgumentParser
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory

from model import get_model

# Define the directories for your train and test data
train_dir = 's3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train'
validation_dir = 's3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test'

def get_train_val_data(train_dir,test_dir):
    test_generator = image_dataset_from_directory(test_dir,
                                              batch_size=32,
                                              image_size = (128,128))


    train_generator = image_dataset_from_directory(train_dir,
                                               batch_size=32,
                                               image_size = (128,128))

    return train_generator, test_generator
    
def parse_args():

    parser = argparse.ArgumentParser()
    
    # Parse input arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--train", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--validation", type=str, default=os.environ["SM_CHANNEL_VALIDATION"])
    parser.add_argument('--learning_rate', type=float, default=0.0001)
    parser.add_argument("--epochs", type=int, default=20)
    parser.add_argument("--batch-size", type=int, default=32)

    return parser.parse_known_args()

if __name__ == "__main__":
    
    args, _ = parse_args()
    
    train_generator, test_generator = get_train_val_data(args.train, args.validation)
    
    model = get_model()
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    model.fit(
            train_generator,
            validation_data = test_generator,
            epochs=20,
            batch_size=32,
            verbose=2
    )

    version = "1"
    model.save(os.path.join(args.model_dir, version)) #model.tar.gz

Overwriting train.py


In [3]:
# Define the directories for your train and test data
train_dir = 's3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/train'
validation_dir = 's3://sagemaker-us-east-1-120644112934/sagemaker/DEMO-tensorflow-chest-scan-image-classification/test'

In [4]:
from sagemaker.tensorflow import TensorFlow

hyperparameters = {"image_size": 128,
                   "batch-size": 32,
                   "epochs": 20
}

estimator = TensorFlow(
                entry_point="train.py",
                source_dir="code",
                role=role,
                framework_version="2.9",
                py_version="py39",
                instance_type="ml.m4.xlarge",  #"ml.t2.medium"
                instance_count=1,
                hyperparameters=hyperparameters
)

inputs = {
    'train': train_dir,
    'validation': validation_dir
}

estimator.fit(inputs)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Miray\AppData\Local\sagemaker\sagemaker\config.yaml
Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2023-10-25-14-49-14-365


2023-10-25 14:50:09 Starting - Starting the training job..
2023-10-25 14:50:33 Starting - Preparing the instances for training....
2023-10-25 14:51:56 Downloading - Downloading input data..
2023-10-25 14:52:29 Training - Downloading the training image..
2023-10-25 14:53:50 Training - Training image download completed. Training in progress.2023-10-25 14:53:57.995453: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-10-25 14:53:57.995664: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-10-25 14:53:58.025315: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-10-25 14:54:00,460 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2023-10-25 14:54:00,471 sagemaker-training-toolkit INFO     No GPUs

15/15 - 14s - loss: 18.6315 - accuracy: 0.5329 - val_loss: 0.6967 - val_accuracy: 0.4667 - 14s/epoch - 915ms/step
Epoch 2/20
15/15 - 12s - loss: 0.6659 - accuracy: 0.5833 - val_loss: 0.7122 - val_accuracy: 0.4667 - 12s/epoch - 811ms/step
Epoch 3/20
15/15 - 12s - loss: 0.6489 - accuracy: 0.6096 - val_loss: 0.6668 - val_accuracy: 0.6222 - 12s/epoch - 803ms/step
Epoch 4/20
15/15 - 12s - loss: 0.6269 - accuracy: 0.6689 - val_loss: 0.6651 - val_accuracy: 0.5778 - 12s/epoch - 801ms/step
Epoch 5/20
15/15 - 13s - loss: 0.5671 - accuracy: 0.6732 - val_loss: 0.5901 - val_accuracy: 0.6111 - 13s/epoch - 878ms/step
Epoch 6/20
15/15 - 12s - loss: 0.5504 - accuracy: 0.7061 - val_loss: 0.5150 - val_accuracy: 0.8000 - 12s/epoch - 800ms/step
Epoch 7/20
15/15 - 12s - loss: 0.4905 - accuracy: 0.7500 - val_loss: 0.4025 - val_accuracy: 0.8333 - 12s/epoch - 803ms/step
Epoch 8/20
15/15 - 12s - loss: 0.3200 - accuracy: 0.8772 - val_loss: 0.3185 - val_accuracy: 0.8556 - 12s/epoch - 803ms/step
Epoch 9/20
15/15 -